In [ ]:
import numpy  as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns


import nltk

import warnings
warnings.filterwarnings('ignore')

import nltk
nltk.download('punkt')

import regex as re

from sklearn.preprocessing import LabelEncoder

from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords


import spacy
from nltk.stem import WordNetLemmatizer



# Scipy
import scipy


# Train-test split and cross validation
from sklearn.model_selection import train_test_split, ParameterGrid

# Classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB        
# from sklearn.naive_bayes import MultinomialNB     
from sklearn.naive_bayes import BernoulliNB     
from sklearn.naive_bayes import CategoricalNB  

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


In [ ]:
## **Problem statement**

In [ ]:
dfsample=pd.read_csv('/content/drive/MyDrive/BW_projects/sample_submission.csv')
dfsample

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
dftrain=pd.read_csv('/content/drive/MyDrive/BW_projects/train.csv')
dftrain.head()


In [ ]:
dftest=pd.read_csv('/content/drive/MyDrive/BW_projects/test.csv')
dftest

Data Cleaning

In [ ]:
dftrain.info()

In [ ]:
dftrain.head(2)

In [ ]:
dftrain.sample(5)

In [ ]:
#checking for missing values
null_counts = dftrain.isnull().sum()
null_percentage = (null_counts / len(dftrain)) * 100
sns.barplot(x=null_percentage.index, y=null_percentage)
plt.xticks(rotation=90)
plt.show()

In [ ]:
dftrain['keyword'] = dftrain['keyword'].fillna('unknown')
dftrain['location'] = dftrain['location'].fillna('unknown')

In [ ]:
print(dftrain[['keyword', 'location']].isnull().sum())

In [ ]:
plt.figure(figsize=(5, 4))
null_counts = dftrain.isnull().sum()
null_percentage = (null_counts / len(dftrain)) * 100
sns.barplot(x=null_percentage.index, y=null_percentage)
plt.xticks(rotation=90)
plt.show()

In [ ]:
#checking for duplicate values
dftrain.duplicated().sum()

In [ ]:
dftrain.shape

In [ ]:
dftest.isnull().sum()

## **EDA**

dftrain['target'].value_counts()

In [ ]:
plt.pie(dftrain['target'].value_counts(), labels=['disaster','not_disaster'],autopct="%0.2f",colors='yc',explode=[0.3,0])
plt.show()

In [ ]:
#checking strenght of characters in each class
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(10,5))
tweet_len= dftrain[dftrain['target']==1]['text'].str.len()
ax1.hist(tweet_len,color='red')
ax1.set_title('disaster tweets')
tweet_len=dftrain[dftrain['target']==0]['text'].str.len()
ax2.hist(tweet_len,color='green')
ax2.set_title('Not disaster tweets')
fig.suptitle('Characters in tweets')
plt.show()

In [ ]:
import nltk
nltk.download('punkt_tab')

In [ ]:
# num of words
dftrain['words_count']=dftrain['text'].apply(lambda x:len(nltk.word_tokenize(x)))

total_words = dftrain['words_count'].sum()
dftrain['words_percentage'] = (dftrain['words_count'] / total_words) * 100

#Spliting into disaster and non-disaster tweets
dftraindisa = dftrain[dftrain['target'] == 1]['words_percentage']
dftrainnondisa = dftrain[dftrain['target'] == 0]['words_percentage']

In [ ]:
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(10,5))
ax1.violinplot(dftraindisa,showmeans=True)
ax1.set_title('disaster tweets')
ax2.violinplot(dftrainnondisa,showmeans=True)
ax2.set_title('Not disaster tweets')
fig.suptitle('words in tweets')
plt.show()